In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("XGBOOST MULTI-CLASS CLASSIFICATION (NOISE-ROBUST)")
print("="*80)

# -----------------------------
# 1. Load data
# -----------------------------
print("\n[1/8] Loading data...")
X = pd.read_csv('trainingData.txt', header=None)
y = pd.read_csv('trainingTruth.txt', header=None, names=['label']).squeeze()
test_data = pd.read_csv('testData.txt', header=None)

print(f"Training data shape: {X.shape}")
print(f"Test data shape: {test_data.shape}")

# -----------------------------
# 2. Data Preprocessing
# -----------------------------
print("\n[2/8] Preprocessing data...")

# Replace empty strings with NaN and convert to numeric
X = X.replace('', np.nan).apply(pd.to_numeric, errors='coerce')
test_data = test_data.apply(pd.to_numeric, errors='coerce')

# Remove rows where y is null
valid_mask = ~y.isna()
X = X[valid_mask].reset_index(drop=True)
y = y[valid_mask].reset_index(drop=True)

print(f"After cleaning: {X.shape[0]} samples, {X.shape[1]} features")

# Analyze missing data
missing_percentage = (X.isna().sum() / len(X)) * 100
features_with_missing = (missing_percentage > 0).sum()
print(f"Features with missing values: {features_with_missing}/{X.shape[1]}")
if features_with_missing > 0:
    print(f"  Max missing %: {missing_percentage.max():.2f}%")
    print(f"  Mean missing %: {missing_percentage[missing_percentage > 0].mean():.2f}%")

# Check class distribution
print("\nClass distribution:")
class_counts = y.value_counts().sort_index()
for cls in class_counts.index:
    print(f"  Class {int(cls)}: {class_counts[cls]} samples ({100*class_counts[cls]/len(y):.2f}%)")

# Check for class imbalance
is_imbalanced = (class_counts.max() / class_counts.min()) > 1.5
if is_imbalanced:
    print("⚠️  Dataset appears imbalanced - will compute class weights")

# Labels to zero-based for XGBoost
y = y - 1

# Calculate class weights if imbalanced
if is_imbalanced:
    class_weights = len(y) / (4 * np.bincount(y.astype(int)))
    sample_weights = np.array([class_weights[int(label)] for label in y])
    print(f"Class weights: {class_weights}")
else:
    sample_weights = None

# -----------------------------
# 3. Imputation
# -----------------------------
print("\n[3/8] Applying median imputation...")

imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)
test_imputed = imputer.transform(test_data)

print(f"Feature count: {X_imputed.shape[1]}")

# -----------------------------
# 4. XGBoost Hyperparameter Configurations
# -----------------------------
print("\n[4/8] Configuring XGBoost parameters (noise-robust)...")

# XGBoost parameters optimized for noisy data:
# - max_depth: Limited depth prevents overfitting to noise
# - min_child_weight: Higher values prevent learning from noisy samples
# - subsample & colsample_bytree: Adds randomness to combat noise
# - gamma: Minimum loss reduction (prevents weak splits)
# - reg_alpha & reg_lambda: L1 and L2 regularization

param_configs = [
    {
        'objective': 'multi:softprob',
        'num_class': 4,
        'max_depth': 7,
        'learning_rate': 0.03,
        'min_child_weight': 30,  # High for noise resistance
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'colsample_bylevel': 0.8,
        'gamma': 0.1,  # Minimum loss reduction
        'reg_alpha': 1.0,  # L1 regularization
        'reg_lambda': 1.5,  # L2 regularization
        'eval_metric': 'mlogloss',
        'seed': 42,
        'tree_method': 'hist',
        'verbosity': 0
    },
    {
        'objective': 'multi:softprob',
        'num_class': 4,
        'max_depth': 6,
        'learning_rate': 0.02,
        'min_child_weight': 40,
        'subsample': 0.75,
        'colsample_bytree': 0.75,
        'colsample_bylevel': 0.75,
        'gamma': 0.15,
        'reg_alpha': 1.5,
        'reg_lambda': 2.0,
        'eval_metric': 'mlogloss',
        'seed': 123,
        'tree_method': 'hist',
        'verbosity': 0
    },
    {
        'objective': 'multi:softprob',
        'num_class': 4,
        'max_depth': 8,
        'learning_rate': 0.04,
        'min_child_weight': 25,
        'subsample': 0.85,
        'colsample_bytree': 0.85,
        'colsample_bylevel': 0.85,
        'gamma': 0.05,
        'reg_alpha': 0.5,
        'reg_lambda': 1.0,
        'eval_metric': 'mlogloss',
        'seed': 456,
        'tree_method': 'hist',
        'verbosity': 0
    },
    {
        'objective': 'multi:softprob',
        'num_class': 4,
        'max_depth': 7,
        'learning_rate': 0.025,
        'min_child_weight': 35,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'colsample_bylevel': 0.8,
        'gamma': 0.12,
        'reg_alpha': 1.2,
        'reg_lambda': 1.8,
        'eval_metric': 'mlogloss',
        'seed': 789,
        'tree_method': 'hist',
        'verbosity': 0
    }
]

# -----------------------------
# 5. Cross-Validation Training
# -----------------------------
print("\n[5/8] Training with 5-fold cross-validation...")

n_folds = 5
skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

all_config_scores = []

for config_idx, params in enumerate(param_configs):
    print(f"\n--- Configuration {config_idx + 1}/{len(param_configs)} ---")
    print(f"LR: {params['learning_rate']}, Depth: {params['max_depth']}, "
          f"Min_child: {params['min_child_weight']}, Gamma: {params['gamma']}")
    
    fold_scores = []
    fold_models = []
    fold_best_iterations = []
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(X_imputed, y)):
        X_train, X_val = X_imputed[train_idx], X_imputed[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        # Create DMatrix with sample weights if available
        if sample_weights is not None:
            dtrain = xgb.DMatrix(X_train, label=y_train, weight=sample_weights[train_idx])
            dval = xgb.DMatrix(X_val, label=y_val, weight=sample_weights[val_idx])
        else:
            dtrain = xgb.DMatrix(X_train, label=y_train)
            dval = xgb.DMatrix(X_val, label=y_val)
        
        # Train model
        evals = [(dval, 'eval')]
        evals_result = {}
        
        model = xgb.train(
            params,
            dtrain,
            num_boost_round=1500,
            evals=evals,
            early_stopping_rounds=100,
            evals_result=evals_result,
            verbose_eval=False
        )
        
        # Validate
        y_val_pred = model.predict(dval)
        y_val_pred_labels = np.argmax(y_val_pred, axis=1)
        accuracy = accuracy_score(y_val, y_val_pred_labels)
        
        fold_scores.append(accuracy)
        fold_models.append(model)
        fold_best_iterations.append(model.best_iteration)
        
        print(f"  Fold {fold + 1}: Accuracy = {accuracy:.4f}, Best iteration = {model.best_iteration}")
    
    # Calculate average CV score
    avg_score = np.mean(fold_scores)
    std_score = np.std(fold_scores)
    avg_best_iter = int(np.mean(fold_best_iterations))
    print(f"  → CV Score: {avg_score:.4f} ± {std_score:.4f}")
    print(f"  → Avg best iteration: {avg_best_iter}")
    
    all_config_scores.append((avg_score, std_score, config_idx, fold_models, avg_best_iter))

# Select best configuration
best_score, best_std, best_config_idx, best_fold_models, best_avg_iter = max(all_config_scores, key=lambda x: x[0])
best_params = param_configs[best_config_idx]

print(f"\n✓ Best configuration: Config {best_config_idx + 1}")
print(f"  CV Score: {best_score:.4f} ± {best_std:.4f}")

# -----------------------------
# 6. Validation Metrics
# -----------------------------
print("\n[6/8] Detailed validation metrics...")

all_val_preds = []
all_val_true = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X_imputed, y)):
    X_val = X_imputed[val_idx]
    y_val = y.iloc[val_idx]
    
    model = best_fold_models[fold]
    dval = xgb.DMatrix(X_val)
    y_val_pred = model.predict(dval)
    
    all_val_preds.append(y_val_pred)
    all_val_true.extend(y_val.values)

all_val_preds = np.vstack(all_val_preds)
all_val_true = np.array(all_val_true)

val_pred_labels = np.argmax(all_val_preds, axis=1)
accuracy = accuracy_score(all_val_true, val_pred_labels)

print(f"\nOverall Validation Accuracy: {accuracy:.4f}")

print("\nClass-wise AUC scores:")
auc_scores = []
for i in range(4):
    y_true_bin = (all_val_true == i).astype(int)
    auc = roc_auc_score(y_true_bin, all_val_preds[:, i])
    auc_scores.append(auc)
    print(f"  Class {i+1} AUC: {auc:.4f}")

macro_auc = np.mean(auc_scores)
print(f"  Macro-average AUC: {macro_auc:.4f}")

print("\nClassification Report:")
print(classification_report(all_val_true, val_pred_labels, 
                          target_names=[f'Class {i+1}' for i in range(4)],
                          digits=4))

# -----------------------------
# 7. Feature Importance
# -----------------------------
print("\n[7/8] Analyzing feature importance...")

# Get feature importance from first model (using gain)
importance = best_fold_models[0].get_score(importance_type='gain')
importance_dict = {int(k.replace('f', '')): v for k, v in importance.items()}

# Create full importance array (including zero-importance features)
full_importance = np.zeros(X_imputed.shape[1])
for idx, val in importance_dict.items():
    full_importance[idx] = val

importance_df = pd.DataFrame({
    'feature_idx': range(len(full_importance)),
    'importance': full_importance
}).sort_values('importance', ascending=False)

print("\nTop 20 most important features:")
for idx, row in importance_df.head(20).iterrows():
    print(f"  Feature {int(row['feature_idx'])}: {row['importance']:.2f}")

zero_importance = (full_importance == 0).sum()
if zero_importance > 0:
    print(f"\n⚠️  {zero_importance} features have zero importance (likely noise)")

# -----------------------------
# 8. Ensemble Prediction on Test Set
# -----------------------------
print("\n[8/8] Generating ensemble predictions on test set...")

ensemble_test_preds = []
seeds = [42, 123, 456, 789, 2024, 2025, 3141, 9876]

for seed_idx, seed in enumerate(seeds):
    print(f"  Training ensemble model {seed_idx + 1}/{len(seeds)} (seed={seed})...")
    
    params_with_seed = best_params.copy()
    params_with_seed['seed'] = seed
    
    # Create DMatrix
    if sample_weights is not None:
        dtrain_full = xgb.DMatrix(X_imputed, label=y, weight=sample_weights)
    else:
        dtrain_full = xgb.DMatrix(X_imputed, label=y)
    
    dtest = xgb.DMatrix(test_imputed)
    
    model = xgb.train(
        params_with_seed,
        dtrain_full,
        num_boost_round=best_avg_iter,
        verbose_eval=False
    )
    
    test_pred = model.predict(dtest)
    ensemble_test_preds.append(test_pred)

# Average ensemble predictions
test_pred_final = np.mean(ensemble_test_preds, axis=0)
test_labels = np.argmax(test_pred_final, axis=1) + 1

# Calculate confidence metrics
prediction_confidence = np.max(test_pred_final, axis=1)
ensemble_agreement = np.array([
    np.mean([np.argmax(pred[i]) == np.argmax(test_pred_final[i]) 
             for pred in ensemble_test_preds])
    for i in range(len(test_pred_final))
])

print(f"\nTest prediction confidence statistics:")
print(f"  Mean: {prediction_confidence.mean():.4f}")
print(f"  Median: {np.median(prediction_confidence):.4f}")
print(f"  Min: {prediction_confidence.min():.4f}")
print(f"  Max: {prediction_confidence.max():.4f}")

print(f"\nTest ensemble agreement statistics:")
print(f"  Mean agreement: {ensemble_agreement.mean():.4f}")
print(f"  High agreement (>0.8): {(ensemble_agreement > 0.8).sum()} samples")
print(f"  Low agreement (<0.5): {(ensemble_agreement < 0.5).sum()} samples")

# Show class distribution
print("\nTest predicted class distribution:")
pred_counts = pd.Series(test_labels).value_counts().sort_index()
for cls in pred_counts.index:
    print(f"  Class {int(cls)}: {pred_counts[cls]} samples ({100*pred_counts[cls]/len(test_labels):.2f}%)")

# Compare with training
print("\nClass distribution comparison (Train → Test):")
for cls in sorted(class_counts.index):
    train_pct = 100 * class_counts[cls] / len(y)
    test_pct = 100 * pred_counts.get(cls, 0) / len(test_labels)
    diff = test_pct - train_pct
    print(f"  Class {int(cls)}: {train_pct:.1f}% → {test_pct:.1f}% (Δ {diff:+.1f}%)")

# -----------------------------
# 9. Save Test Results
# -----------------------------
output = np.column_stack([test_pred_final, test_labels])
np.savetxt('testLabel_xgboost.txt', output, 
           fmt='%.6f\t%.6f\t%.6f\t%.6f\t%d', 
           delimiter='\t')

np.savetxt('testLabel_xgboost_confidence.txt', 
           np.column_stack([test_labels, prediction_confidence, ensemble_agreement]),
           fmt='%d\t%.6f\t%.6f',
           header='predicted_label\tconfidence\tensemble_agreement',
           comments='')

print(f"\n✓ Test predictions saved to 'testLabel_xgboost.txt'")
print(f"✓ Confidence metrics saved to 'testLabel_xgboost_confidence.txt'")

# -----------------------------
# 10. Blind Data Prediction
# -----------------------------
print("\n" + "="*80)
print("[9/9] Generating predictions for BLIND dataset...")
print("="*80)

try:
    blind_data = pd.read_csv('blindData.txt', header=None)
    print(f"\nBlind data shape: {blind_data.shape}")
    
    blind_data = blind_data.apply(pd.to_numeric, errors='coerce')
    blind_imputed = imputer.transform(blind_data)
    
    print(f"Preprocessed blind data shape: {blind_imputed.shape}")
    print(f"\nGenerating ensemble predictions with {len(seeds)} models...")
    
    ensemble_blind_preds = []
    
    for seed_idx, seed in enumerate(seeds):
        print(f"  Model {seed_idx + 1}/{len(seeds)}...", end='\r')
        
        params_with_seed = best_params.copy()
        params_with_seed['seed'] = seed
        
        if sample_weights is not None:
            dtrain_full = xgb.DMatrix(X_imputed, label=y, weight=sample_weights)
        else:
            dtrain_full = xgb.DMatrix(X_imputed, label=y)
        
        dblind = xgb.DMatrix(blind_imputed)
        
        model = xgb.train(
            params_with_seed,
            dtrain_full,
            num_boost_round=best_avg_iter,
            verbose_eval=False
        )
        
        blind_pred = model.predict(dblind)
        ensemble_blind_preds.append(blind_pred)
    
    print(f"  Completed all {len(seeds)} models    ")
    
    # Average predictions
    blind_pred_final = np.mean(ensemble_blind_preds, axis=0)
    blind_labels = np.argmax(blind_pred_final, axis=1) + 1
    
    # Calculate metrics
    blind_confidence = np.max(blind_pred_final, axis=1)
    blind_agreement = np.array([
        np.mean([np.argmax(pred[i]) == np.argmax(blind_pred_final[i]) 
                 for pred in ensemble_blind_preds])
        for i in range(len(blind_pred_final))
    ])
    
    print(f"\nBlind prediction confidence statistics:")
    print(f"  Mean: {blind_confidence.mean():.4f}")
    print(f"  Median: {np.median(blind_confidence):.4f}")
    print(f"  Min: {blind_confidence.min():.4f}")
    print(f"  Max: {blind_confidence.max():.4f}")
    
    print(f"\nBlind ensemble agreement statistics:")
    print(f"  Mean agreement: {blind_agreement.mean():.4f}")
    print(f"  High agreement (>0.8): {(blind_agreement > 0.8).sum()} samples")
    print(f"  Low agreement (<0.5): {(blind_agreement < 0.5).sum()} samples")
    
    print("\nBlind predicted class distribution:")
    blind_pred_counts = pd.Series(blind_labels).value_counts().sort_index()
    for cls in blind_pred_counts.index:
        print(f"  Class {int(cls)}: {blind_pred_counts[cls]} samples ({100*blind_pred_counts[cls]/len(blind_labels):.2f}%)")
    
    print("\nClass distribution comparison (Train → Test → Blind):")
    for cls in sorted(class_counts.index):
        train_pct = 100 * class_counts[cls] / len(y)
        test_pct = 100 * pred_counts.get(cls, 0) / len(test_labels)
        blind_pct = 100 * blind_pred_counts.get(cls, 0) / len(blind_labels)
        print(f"  Class {int(cls)}: {train_pct:.1f}% → {test_pct:.1f}% → {blind_pct:.1f}%")
    
    # Save predictions
    blind_output = np.column_stack([blind_pred_final, blind_labels])
    np.savetxt('blindLabel_xgboost.txt', blind_output, 
               fmt='%.6f\t%.6f\t%.6f\t%.6f\t%d', 
               delimiter='\t')
    
    np.savetxt('blindLabel_xgboost_confidence.txt', 
               np.column_stack([blind_labels, blind_confidence, blind_agreement]),
               fmt='%d\t%.6f\t%.6f',
               header='predicted_label\tconfidence\tensemble_agreement',
               comments='')
    
    print(f"\n✓ Blind predictions saved to 'blindLabel_xgboost.txt'")
    print(f"✓ Blind confidence metrics saved to 'blindLabel_xgboost_confidence.txt'")
    print(f"  - {len(blind_labels)} predictions generated")
    
except FileNotFoundError:
    print("\n⚠️  'blindData.txt' not found - skipping blind data prediction")
except Exception as e:
    print(f"\n⚠️  Error processing blind data: {e}")

# -----------------------------
# Final Summary
# -----------------------------
print("\n" + "="*80)
print("✓ ALL PREDICTIONS COMPLETED SUCCESSFULLY")
print("="*80)
print(f"\nFiles generated:")
print(f"  1. testLabel_xgboost.txt ({len(test_labels)} predictions)")
print(f"  2. testLabel_xgboost_confidence.txt (test confidence scores)")
try:
    print(f"  3. blindLabel_xgboost.txt ({len(blind_labels)} predictions)")
    print(f"  4. blindLabel_xgboost_confidence.txt (blind confidence scores)")
except:
    pass

print(f"\nModel Performance Summary:")
print(f"  - Algorithm: XGBoost (Noise-Robust)")
print(f"  - Ensemble size: {len(seeds)} models")
print(f"  - Expected validation accuracy: {best_score:.4f} ± {best_std:.4f}")
print(f"  - Expected macro-average AUC: {macro_auc:.4f}")
print(f"  - Best configuration: Config {best_config_idx + 1}")
print(f"  - Noise resistance features: gamma, min_child_weight, regularization")
print("\n" + "="*80)

XGBOOST MULTI-CLASS CLASSIFICATION (NOISE-ROBUST)

[1/8] Loading data...
Training data shape: (27617, 411)
Test data shape: (13082, 411)

[2/8] Preprocessing data...
After cleaning: 27617 samples, 411 features
Features with missing values: 1/411
  Max missing %: 0.50%
  Mean missing %: 0.50%

Class distribution:
  Class 1: 8874 samples (32.13%)
  Class 2: 6127 samples (22.19%)
  Class 3: 8483 samples (30.72%)
  Class 4: 4133 samples (14.97%)
⚠️  Dataset appears imbalanced - will compute class weights
Class weights: [0.77803133 1.12685654 0.81389249 1.67051778]

[3/8] Applying median imputation...
Feature count: 411

[4/8] Configuring XGBoost parameters (noise-robust)...

[5/8] Training with 5-fold cross-validation...

--- Configuration 1/4 ---
LR: 0.03, Depth: 7, Min_child: 30, Gamma: 0.1
  Fold 1: Accuracy = 0.9155, Best iteration = 1499
  Fold 2: Accuracy = 0.9003, Best iteration = 1499
  Fold 3: Accuracy = 0.9136, Best iteration = 1499
  Fold 4: Accuracy = 0.9098, Best iteration = 1